Загружаю датасет

In [27]:
from config import *

In [2]:
import pandas as pd

df = pd.read_csv(DATASET_BWSAS_FILE_ABS_PATH)
df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True) # Replace spaces with underscores in column names
df = df.convert_dtypes()
df.Measurement_Timestamp = pd.to_datetime(df.Measurement_Timestamp)
df.dtypes


C:\Users\MANDR\AppData\Local\Temp\ipykernel_18420\1903369583.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.Measurement_Timestamp = pd.to_datetime(df.Measurement_Timestamp)


Station_Name                   string[python]
Measurement_Timestamp          datetime64[ns]
Air_Temperature                       Float64
Wet_Bulb_Temperature                  Float64
Humidity                                Int64
Rain_Intensity                        Float64
Interval_Rain                         Float64
Total_Rain                            Float64
Precipitation_Type                      Int64
Wind_Direction                          Int64
Wind_Speed                            Float64
Maximum_Wind_Speed                    Float64
Barometric_Pressure                   Float64
Solar_Radiation                         Int64
Heading                                 Int64
Battery_Life                          Float64
Measurement_Timestamp_Label    string[python]
Measurement_ID                 string[python]
dtype: object

Динамически создаю .proto файл для него (не работает)

In [3]:
import pandas as pd
from google.protobuf.descriptor_pb2 import FieldDescriptorProto
from google.protobuf.descriptor_pb2 import FileDescriptorProto
from google.protobuf.compiler.plugin_pb2 import CodeGeneratorRequest
from google.protobuf.compiler.plugin_pb2 import CodeGeneratorResponse

def create_protobuf_template(df):
    # Create a FileDescriptorProto
    file_descriptor_proto = FileDescriptorProto()

    # Create a message type for the DataFrame
    message_type = file_descriptor_proto.message_type.add()
    message_type.name = "DataFrame"

    # Iterate over the columns of the DataFrame
    for column_name, column_type in df.dtypes.items():
        # Create a field descriptor for each column
        field_descriptor = message_type.field.add()
        field_descriptor.name = column_name

        # Map Pandas dtype to Protobuf field type
        if column_type == "int64":
            field_descriptor.type = FieldDescriptorProto.TYPE_INT64
        elif column_type == "float64":
            field_descriptor.type = FieldDescriptorProto.TYPE_DOUBLE
        elif column_type == "bool":
            field_descriptor.type = FieldDescriptorProto.TYPE_BOOL
        elif column_type == "object":
            field_descriptor.type = FieldDescriptorProto.TYPE_STRING
        elif column_type == "datetime64[ns]":
            field_descriptor.type = FieldDescriptorProto.TYPE_STRING
        else:
            field_descriptor.type = FieldDescriptorProto.TYPE_STRING

    # Return the serialized FileDescriptorProto
    return file_descriptor_proto.SerializeToString()

In [4]:
protobuf_template = create_protobuf_template(df)
print(protobuf_template)

b'"\x82\x03\n\tDataFrame\x12\x10\n\x0cStation_Name(\t\x12\x19\n\x15Measurement_Timestamp(\t\x12\x13\n\x0fAir_Temperature(\t\x12\x18\n\x14Wet_Bulb_Temperature(\t\x12\x0c\n\x08Humidity(\t\x12\x12\n\x0eRain_Intensity(\t\x12\x11\n\rInterval_Rain(\t\x12\x0e\n\nTotal_Rain(\t\x12\x16\n\x12Precipitation_Type(\t\x12\x12\n\x0eWind_Direction(\t\x12\x0e\n\nWind_Speed(\t\x12\x16\n\x12Maximum_Wind_Speed(\t\x12\x17\n\x13Barometric_Pressure(\t\x12\x13\n\x0fSolar_Radiation(\t\x12\x0b\n\x07Heading(\t\x12\x10\n\x0cBattery_Life(\t\x12\x1f\n\x1bMeasurement_Timestamp_Label(\t\x12\x12\n\x0eMeasurement_ID(\t'


In [6]:
# Write the byte string to a file.
with open(path.join(PROTOS_FOLDER_ABS_PATH, "protobuf_example.proto"), "wb") as f:
    f.write(protobuf_template)

Компилирую .proto файл

In [29]:
import os

M_BEACH_PROTO = path.join(PROTOS_FOLDER, "m_beach.proto")

compile_proto_command = f"{PROTOC_EXE} --python_out=. {M_BEACH_PROTO}"
os.system(compile_proto_command)

NameError: name 'PROTOC_EXE' is not defined

Заполняю сообщение данными

In [8]:
def timestamp2unix(timestamp):
    seconds = int(timestamp.timestamp())
    nanos = int(timestamp.timestamp() % 1 * 1e9)

    return (seconds, nanos)

In [9]:
import protos.m_beach_pb2 as m_beach_pb2

m_beach_message = m_beach_pb2.m_beach_message() 

for field, value in df.to_dict(orient='records')[0].items():
    if value is None:
        continue
    if type(value) is pd.Timestamp:
        seconds, nanos = timestamp2unix(value)
        setattr(getattr(m_beach_message, field), "seconds", seconds)
        setattr(getattr(m_beach_message, field), "nanos", nanos)
    else:
        try:
            setattr(m_beach_message, field, value)
        except Exception as e:
            print(f"Exception: {e} | field: {field} | value: {value}")

print(m_beach_message)


Station_Name: "Oak Street Weather Station"
Measurement_Timestamp {
  seconds: 1432306800
}
Wet_Bulb_Temperature: 7
Humidity: 55
Total_Rain: 1.4
Wind_Direction: 63
Wind_Speed: 1.9
Maximum_Wind_Speed: 2.8
Solar_Radiation: 780
Heading: 322
Battery_Life: 12
Measurement_Timestamp_Label: "05/22/2015 3:00 PM"
Measurement_ID: "OakStreetWeatherStation201505221500"



Сериализую и десериализую

In [12]:
with open("m_beach.bin", "wb") as f:
    print("write as binary")
    bytesAsString = m_beach_message.SerializeToString()
    f.write(bytesAsString)


with open("m_beach.bin", "rb") as f:
    print("read values")
    simple_message_read = m_beach_pb2.m_beach_message().FromString(f.read())


print(simple_message_read)

write as binary
read values
Station_Name: "Oak Street Weather Station"
Measurement_Timestamp {
  seconds: 1432306800
}
Wet_Bulb_Temperature: 7
Humidity: 55
Total_Rain: 1.4
Wind_Direction: 63
Wind_Speed: 1.9
Maximum_Wind_Speed: 2.8
Solar_Radiation: 780
Heading: 322
Battery_Life: 12
Measurement_Timestamp_Label: "05/22/2015 3:00 PM"
Measurement_ID: "OakStreetWeatherStation201505221500"



<hr>

ВСЯКОЕ

In [14]:
m_beach_message.DESCRIPTOR.__dir__()

['__new__',
 'GetOptions',
 'CopyToProto',
 'EnumValueName',
 'name',
 'full_name',
 '_concrete_class',
 'file',
 'fields',
 'fields_by_name',
 'fields_by_camelcase_name',
 'fields_by_number',
 'nested_types',
 'nested_types_by_name',
 'extensions',
 'extensions_by_name',
 'extension_ranges',
 'enum_types',
 'enum_types_by_name',
 'enum_values_by_name',
 'oneofs_by_name',
 'oneofs',
 'containing_type',
 'is_extendable',
 'has_options',
 'syntax',
 '__doc__',
 '__module__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__init__',
 '__reduce_ex__',
 '__reduce__',
 '__getstate__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [19]:
print([field.name for field in m_beach_message.DESCRIPTOR.fields])


['Station_Name', 'Measurement_Timestamp', 'Air_Temperature', 'Wet_Bulb_Temperature', 'Humidity', 'Rain_intensity', 'interval_Rain', 'Total_Rain', 'Precipitation_Type', 'Wind_Direction', 'Wind_Speed', 'Maximum_Wind_Speed', 'Barometric_Pressure', 'Solar_Radiation', 'Heading', 'Battery_Life', 'Measurement_Timestamp_Label', 'Measurement_ID']


In [23]:
df.to_dict(orient='records')[0]

{'Station_Name': 'Oak Street Weather Station',
 'Measurement_Timestamp': '05/22/2015 03:00:00 PM',
 'Air_Temperature': None,
 'Wet_Bulb_Temperature': 7.0,
 'Humidity': 55,
 'Rain_Intensity': 0.0,
 'Interval_Rain': 0.0,
 'Total_Rain': 1.4,
 'Precipitation_Type': 0,
 'Wind_Direction': 63,
 'Wind_Speed': 1.9,
 'Maximum_Wind_Speed': 2.8,
 'Barometric_Pressure': None,
 'Solar_Radiation': 780,
 'Heading': 322,
 'Battery_Life': 12.0,
 'Measurement_Timestamp_Label': '05/22/2015 3:00 PM',
 'Measurement_ID': 'OakStreetWeatherStation201505221500'}